# Combining our tools with pandas

This is a quick assignment meant to show you how to make pandas work together with other tools.

* **SQL** is great for large datasets
* **Python** is great for poking around at data
* **Command line tools** are great for quick analysis
* **Excel** is easily shareable

### Before we start

Import pandas the right way.

In [3]:
import pandas as pd

## Postgres to pandas: using `read_sql` in pandas

Hm, problem 2 of one of your SQL assignments seems interesting.

> Using avg(), get the average solar energy usage for each country, and display only the countries with an average that is above 500 (million). Make sure it is properly ordered like the results below, and wrap your avg column in round() to get round numbers:
> 
>
> |avg solar energy|country|
> |---|---|
> |91339.0|China|
> |20425.0|India|
> |14084.0|United States|
> |12142.0|Germany|
> |...|...|

Use `pd.read_sql` to run the query against your Postgres database and return the result as a pandas dataframe. While you can use `pg8000`, I think SQLAlchemy is the more common method for connecting to a database.

* **TIP:**  [This page might help](https://towardsdatascience.com/work-with-sql-in-python-using-sqlalchemy-and-pandas-cd7693def708)
* **TIP:** If you didn't get the answer to that homework, just.... run anything, I guess.

In [2]:
import pg8000
# use whatever username information you used in classed to connect
conn = pg8000.connect(database="undata",user="minjukim")
print(type(conn))

<class 'pg8000.legacy.Connection'>


In [5]:
cursor = conn.cursor()
myquery = '''
SELECT country, ROUND(avg(usage))
FROM solar
GROUP BY country
HAVING avg(usage) > 500
ORDER BY avg(usage) DESC NULLS LAST;
'''
# cursor.execute(myquery)

pd.read_sql(myquery, conn)

/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_53124/3524318623.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(myquery, conn)


,country,round
0,China,91339.0
1,India,20425.0
2,United States,14084.0
3,Germany,12142.0
4,Japan,11094.0
5,Italy,6414.0
6,Spain,4488.0
7,Turkey,3532.0
8,United Kingdom,2992.0
9,Brazil,2733.0


## CSV to SQLite: using `to_sql` in pandas

SQLite is like a baby version of Postgres. It's super convenient because it just creates a file instead of needing a big giant server!

Even if you used `pg8000` for the last one, you should probably level up to SQLAlchemy. [Here are notes on connections](https://docs.sqlalchemy.org/en/13/core/connections.html), and [here's how `.to_sql` works](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html).

In [6]:
!pip install sqlalchemy


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


### Step 1: Create a SQLAlchemy "engine" (a connection) for a SQLite database you'll call `world.db`

* **TIP:** The filename format for a sqlite connection is `sqlite:///filename.db` – that's *three forward slashes*. More and you'll get in trouble.

In [9]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///world.db', echo=False)

### Step 2: Read `countries.csv` and `continents.csv` in as separate dataframes

In [20]:
df_countries = pd.read_csv("countries.csv")
df_continents = pd.read_csv("continent_facts.csv")

### Step 3: Using `.to_sql`

Using your SQLAlchemy connection to `world.db`, save the countries data to a table called "countries" and the continent data to a table called "continents"

**If the table already exists, it should be replaced.**

* **TIP:** As stated in the headline, you will be using `.to_sql`
* **TIP:** Use `index=False` when saving, or else the index will for some reason get its own column.
* **TIP:** Shift+tab will give you documentation! The connection parameter is `con=` and not `conn=` because nothing in pandas can ever be nice.

In [18]:
df_countries.to_sql('countries', con=engine, index=False, if_exists='replace')

188

In [21]:
df_continents.to_sql('continents', con=engine, index=False, if_exists='replace')

7

### Step 4: Using `.read_sql`

Use `pd.read_sql` and your sqlite engine to run a query on your database and save the result into a pandas dataframe. I've given you your query below:

```python
query = """
SELECT * FROM countries
JOIN continents ON countries.continent = continents.continent
"""
```

The result should be a dataframe that includes the columns country, continent, life expectancy, population, pdf, continent, and square miles. It should only have ONE index, on the far left hand side, bolded. If it has multiples then you need to add `index=False` to the previous commands.

In [29]:
query = """
SELECT * FROM countries
JOIN continents ON countries.continent = continents.continent
"""

df = pd.read_sql(query, con=engine)

In [30]:
df

,country,continent,life_expectancy,population,gdp,continent,sqmi_mil
0,Afghanistan,Asia,54.863,22856302,15153728226,Asia,17.210
1,Albania,Europe,74.200,3071856,12886435920,Europe,3.931
2,Algeria,Africa,68.963,30533827,155661450046,Africa,11.730
3,Angola,Africa,45.234,13926373,34063908358,Africa,11.730
4,Armenia,Europe,71.494,3076098,6502871172,Europe,3.931
...,...,...,...,...,...,...,...
149,Vietnam,Asia,73.777,78758010,124201381770,Asia,17.210
150,West Bank and Gaza,Asia,70.929,3198560,24264276160,Asia,17.210
151,"Yemen, Rep.",Asia,60.404,17723186,39292303362,Asia,17.210
152,Zambia,Africa,41.802,10201562,10558616670,Africa,11.730


Why sqlite? ...because I don't have postgres installed here and I don't want to accidentally set something up wrong.

## CSVkit to SQLite

I dunno, that seemed like a lot of work. Can't I just use like CSVKit to something to shove all the data into a database?

Using [this as reference](https://gist.github.com/martijngastkemper/6d62245565a49081db6024f5f432fe63), use the `csvsql` query to insert the data from `worldcities.csv` into `world.db`. Have it insert into a table called `cities`, and overwrite the table if it already exists.

In [32]:
!csvsql --db sqlite:///world.db --table cities --insert worldcities.csv

Test if it worked by successfully running the following using `.read_sql`.

```
SELECT
    cities.city, countries.country, cities.population,
    ROUND(cities.population / countries.population * 100) AS pct_pop
FROM cities
JOIN countries ON cities.country = countries.country
ORDER BY pct_pop DESC
LIMIT 10
```

It should calculate percent of the country's population each city is, but it... the data is bad. It's a data issue.

In [33]:
query = '''
SELECT
    cities.city, countries.country, cities.population,
    ROUND(cities.population / countries.population * 100) AS pct_pop
FROM cities
JOIN countries ON cities.country = countries.country
ORDER BY pct_pop DESC
LIMIT 10
'''

pd.read_sql(query, con=engine)

,city,country,population,pct_pop
0,Doha,Qatar,1186023.0,201.0
1,Kuwait City,Kuwait,2989000.0,154.0
2,Singapore,Singapore,5271000.0,134.0
3,Amman,Jordan,4007526.0,83.0
4,Dubai,United Arab Emirates,2502715.0,83.0
5,Djibouti,Djibouti,562000.0,77.0
6,Libreville,Gabon,797003.0,65.0
7,Luanda,Angola,8883000.0,64.0
8,Muscat,Oman,1421409.0,63.0
9,Ulaanbaatar,Mongolia,1396288.0,58.0


## Reading Google Sheets from pandas

I was trying to find a good spreadsheet for you to look at, and for some reason sitting at the top of my Google Drive was [this random list of anime and various scores and things](https://docs.google.com/spreadsheets/d/1CV9_bm7LrrjlDNb12ftBZBolAhtfunZP8j9g1sxbDFM/edit#gid=77782877). Why? I have no idea. Seems official, though.

### Read in the `Most favorited` spreadsheet and look at the first five

> **TIP:** [Use this walkthrough](https://towardsdatascience.com/read-data-from-google-sheets-into-pandas-without-the-google-sheets-api-5c468536550), but beware of fancy quotes

In [41]:
sheet_id = "1CV9_bm7LrrjlDNb12ftBZBolAhtfunZP8j9g1sxbDFM"
sheet_name = "403834772" 
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&gid={sheet_name}"

most_favorited = pd.read_csv(url)

most_favorited.head(5)



,Rank,Anime Name,Favorited %,MAL Score,MAL Rank,Rank Difference
0,1,Ginga Eiyuu Densetsu,30.2%,9.09,9,8▲
1,2,Gintama,27.9%,9.05,10,8▲
2,3,One Piece,19.4%,8.61,61,58▲
3,4,Hunter x Hunter (2011),18.8%,9.15,5,1▲
4,5,Steins;Gate,18.3%,9.17,2,3▼


### Read in the `Most Dropped` spreadsheet and look at the first 5

What was the difference between the two? The URL is *mostly* the same.

In [42]:
sheet_id = "1CV9_bm7LrrjlDNb12ftBZBolAhtfunZP8j9g1sxbDFM"
dropped_sheet_name = "77782877" 
dropped_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&gid={dropped_sheet_name}"

most_dropped = pd.read_csv(dropped_url)
most_dropped.head()



,Rank,Anime Name,Dropped %,MAL Score,MAL Rank,Unnamed: 5
0,1,Vampire Holmes,31.2%,3.70,7171,NaN
1,2,Gifuu Doudou!!: Kanetsugu to Keiji,29.6%,6.55,4843,NaN
2,3,Majin Bone,29.3%,6.89,3590,NaN
3,4,Abarenbou Rikishi!! Matsutarou,28.8%,6.20,5827,NaN
4,5,Tottoko Hamtarou,26.6%,7.05,3103,NaN


## Reading HTML tables with pandas

This is the real killer app here. `pd.read_html` doesn't really read HTML, it reads *HTML tables*.

Use `pd.read_html` to read in the big watermelons from [here](http://www.bigpumpkins.com/WeighoffResultsGPC.aspx?c=W&y=2022). Look at the first three.

* **TIP:** Read HTML gives you a *list of all of the HTML tables on the page*. This means "the output is going to look ugly, but it's not a Series, it's a list of dataframes."
* **TIP:** I think you should save that as a variable called `tables`
* **TIP:** Then just keep looking at the first table, the second, the third, with `[0]`, `[1]` etc, until you find the one you want. Then save that as `df`.

In [36]:
tables = pd.read_html("http://www.bigpumpkins.com/WeighoffResultsGPC.aspx?c=W&y=2022")

In [37]:
tables[0]

,Place,Weight (lbs),Grower Name,City,State/Prov,Country,GPC Site,Seed (Mother),Pollinator (Father),OTT,Est. Weight,Pct. Chart
0,1,325.40,"Mudd, Framk",Vine Grove,Kentucky,United States,Allardt Pumpkin Festival,305 Mudd 16,305 Mudd,223.0,303.00,7.0
1,2,309.00,"McCaslin, Nick",Hawesville,Kentucky,United States,Chillicothe Halloween Festival,301.5 McCaslin,Self,224.0,307.00,1.0
2,3,306.00,"Vial, Andrew",Liberty,North Carolina,United States,NC State Fair GPC Weigh-Off,341.5 Vial 19,330.5 Vial B 19,223.0,301.00,2.0
3,4,302.50,"Mudd, Frank",Vine Grove,Kentucky,United States,Roberts Family Farms,305 Mudd 16,Self,221.0,297.00,2.0
4,5,291.50,"VanBeck, Patrick",Willlow Spring,North Carolina,United States,NC State Fair GPC Weigh-Off,Carolina Cross Burpee,305 Vial DMG,221.0,297.00,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
302,NaN,This page shows the GPC results for the select...,This page shows the GPC results for the select...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,NaN,PlaceWeight (lbs)Grower NameCityState/ProvCoun...,PlaceWeight (lbs)Grower NameCityState/ProvCoun...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305,Top of Page Questions or comments? Send mail t...,Top of Page Questions or comments? Send mail t...,Top of Page Questions or comments? Send mail t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
